In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
def extraction(year,target):
  print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
  print(year,target)
  path = '/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/fct_weather_data/'+str(year)
  area_list = os.listdir(path)

  area_dic ={key:list() for key in area_list}
  for area in area_list:
    file_list = os.listdir(path+'/'+area+'/')
    for file in file_list:
      if target in file:
        res = pd.read_csv(path+'/'+area+'/'+file,encoding='cp949')
        area_dic[area].append(res)


  for area in area_list:
    length = len(area_dic[area])
    for i in range(length):

      # 컬럼명 변경
      df=area_dic[area][i] # 약한 복사 (원본 데이터도 바뀜, 이름 편하게 쓰려고)
      df.columns = ['day','hour','forecast','val']

      df.drop_duplicates(inplace=True)

      # 'day' 컬럼에 Start 란 문자열이 들어간 행(월 구분 행) index 추출 
      idxs = df[df['day'].str.contains('Start')].index


      # 'month' 컬럼 추가 
      start_num = 0
      month = 1 
      df['month'] = 0
      for end_num in idxs:
        df.loc[start_num:end_num,'month']= str(month)
        month += 1
        start_num = end_num
      df.loc[start_num:,'month']= month
      del month


      # 'day' 컬럼에 Start 란 문자열이 들어간 행 삭제
      df.drop(idxs,axis=0,inplace=True)


      if(df['hour'].dtypes != 'float'):
        idx = df[df['hour']=='hour'].index
        df.drop(idx,axis=0,inplace=True)


      # 'hour' 컬럼은 현재 float 형(오전 2시는 200, 오후 2시는 1400 이래 표기중)
      # 'hour' 을 100으로 나눠서 최대 10의 자리 정수로 변환
      df['hour'] = df['hour'].astype(float)
      df['hour'] = df['hour']//100



      df['forecast'] = df['forecast'].astype('float')
      # 'hour' 과 'forecast' 의 합이 9, 12, 15, 33, 36, 39 인 행만 추출 살림
      condition1 = ((df['hour']+df['forecast'])!=9)
     # condition2 = ((df['hour']+df['forecast'])!=12)
      condition3 = ((df['hour']+df['forecast'])!=15)
      condition4 = ((df['hour']+df['forecast'])!=18)
   #   condition5 = ((df['hour']+df['forecast'])!=36)
    #  condition6 = ((df['hour']+df['forecast'])!=39)
      drop_idxs = df.loc[
                        condition1 &
                        # condition2 &
                         condition3 &
                         condition4 
                         #condition5 &
                         #condition6
                         ].index
      df.drop(drop_idxs,axis=0,inplace =True)
        

      # 'when'컬럼 추가 예보 대상 시점 정보를 담음
      # 예보시점은 불필요
      df['when'] = df['hour']+df['forecast']
      df.drop('hour',axis=1,inplace =True)

      new_columns = ['month','day','when','forecast','val']
      area_dic[area][i] = df[new_columns]
      del new_columns
      # 달,일,예보대상시점,몇시간전 예보인지 순으로 재정렬 & 인덱스 재설정
      df.sort_values(['month','day','when','forecast'],inplace=True)
      df.reset_index(drop=True, inplace=True)


  # regional_integration_df 딕셔너리 안에 지역별 outer join 된 dataframe이 들어감
  # 딕셔너리 키 값은 지역명
  regional_integration_df = {key:0 for key in area_list}


  for area in area_list:
    length = len(area_dic[area])
    if length == 0:
      continue
    res = area_dic[area][0].copy()
    for i in range(1,length):
      res = pd.merge(res,area_dic[area][i].copy(),how='outer',on=['month','day','when','forecast'])
    regional_integration_df[area]=res

  # 관측소마다 내놓은 예보 값의 mean과 max를 구함
  for area in area_list: 
    regional_integration_df[area].iloc[:,4:] = regional_integration_df[area].iloc[:,4:].astype('float')
    val_mean = regional_integration_df[area].iloc[:,4:].mean(axis='columns')
    # val_max = regional_integration_df[area].iloc[:,4:].max(axis=1)
    regional_integration_df[area][f'{target}_mean'] = val_mean
    # regional_integration_df[area][f'{target}_max'] = val_max

    # 'yyyymmdd'컬럼 생성 (datetime 형식) 
    regional_integration_df[area]['year'] = year
    regional_integration_df[area]['month'] = regional_integration_df[area]['month'].astype(int) 
    regional_integration_df[area]['day'] = regional_integration_df[area]['day'].astype(int)
    regional_integration_df[area]['yyyymmdd'] = regional_integration_df[area]['year']*10000 + regional_integration_df[area]['month']*100+regional_integration_df[area]['day'] 
    regional_integration_df[area].drop(['month','day','year'],axis = 1, inplace=True)
    regional_integration_df[area]['yyyymmdd'] = regional_integration_df[area]['yyyymmdd'].astype(str)
      
    # 사용할 컬럼만 추림 (['yyyymmdd''when','forecast','val_mean','val_max'])
    regional_integration_df[area] = regional_integration_df[area][['yyyymmdd','when','forecast',f'{target}_mean',
                                                                    #  f'{target}_max'
                                                                    ]]

    regional_integration_df[area]['yyyymmdd'] = pd.to_datetime(regional_integration_df[area]['yyyymmdd'], dayfirst=True, errors='coerce')
    regional_integration_df[area] = regional_integration_df[area][pd.notnull(regional_integration_df[area]['yyyymmdd'])]

    # 병병원데이터에 붙힐 수 있도록 reshape
    regional_integration_df[area]['yyyymmdd'] = pd.to_datetime(regional_integration_df[area]['yyyymmdd'])
    regional_integration_df[area]  = regional_integration_df[area].groupby(['yyyymmdd','when'],as_index=False)[f'{target}_mean',
                                                                                                              #  f'{target}_max'
                                                                                                              ].mean()
    regional_integration_df[area] = regional_integration_df[area].pivot(index='yyyymmdd',columns=['when'],values=[f'{target}_mean',
                                                                                                                  # f'{target}_max'
                                                                                                                  ])
    regional_integration_df[area].columns = [col[0]+'_'+str(int(col[1])) for col in regional_integration_df[area].columns]

    # 지역명도 column에 추가
    print(area)
    regional_integration_df[area]['area'] = area


  return regional_integration_df

In [ ]:
from tqdm import tqdm 
area_list = ['전북',
 '제주',
 '충남',
 '충북',
 '강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남']


subject = [
    '3시간기온',
           '습도',
           '일최고기온',
           '일최저기온',
           '풍속',
           '강수량',
           '적설'
           ]

fct_final = []

for sub in tqdm(subject):
  
  res_sep_year = []
  res_df = pd.DataFrame()

  for year in range(2012,2016+1):
    # extraction() 에 연도와 목표 data의 이름을 넣으면 지역별로 정리된 dictionary 반환
    # res_sep_year 는 시간별로 정리된 dictionay list (인덱스 0 은 2012 , 인덱스 5는 2016)
    
    res_sep_year.append(extraction(year,sub))
    
    for area in area_list:
      # 각 지역별로 내일 예보는 한칸씩 내림
      res_sep_year[year-2012][area].iloc[:,3:6] = res_sep_year[year-2012][area].iloc[:,3:6].shift(periods = 1 ,axis = 0)

      #시계열로 데이터 보간
      res_sep_year[year-2012][area].iloc[:,0:6] = res_sep_year[year-2012][area].iloc[:,0:6].interpolate(method='time')
      res_sep_year[year-2012][area].iloc[:,0:6] = res_sep_year[year-2012][area].iloc[:,0:6].interpolate(method='time',limit_direction='backward')


      # 지역별로 나눠진 df 하나로 합치기 (pre_merge)
      res_df = pd.concat([res_df,res_sep_year[year-2012][area]],axis = 0)

  # 시계열 인덱스로부터 'yyyymmdd' 칼럼 추출
  res_df['yyyymmdd'] = res_df.index
  # 'yyyymmdd'를 back의 'yyyymmdd' 형식과 일치시킴
  res_df['yyyymmdd'] = res_df['yyyymmdd'].astype(str)
  res_df['yyyymmdd'] = res_df['yyyymmdd'].str.replace(pat=r'-', repl=r'', regex=True)
  res_df['yyyymmdd'] = res_df['yyyymmdd'].astype(int)
  # res 인덱스를 일반적인 색인으로 변환 (0,1,2,3,...) 
  res_df.index = [i for i in range(len(res_df))]  

  fct_final.append(res_df)

# 주제별로 나눠진 df 하나로 합치기 (pre_merge)
pre_merge = pd.merge(fct_final[0], fct_final[1], how='outer', on =['yyyymmdd','area'])
for i in range(2,7):
  pre_merge = pd.merge(pre_merge, fct_final[i], how='outer', on =['yyyymmdd','area'])
  
# # 백병원데이터
# back = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_res.csv',encoding='cp949')
# back.drop(back.columns[4:],axis=1,inplace=True)

# # 이거 back['area'] 랑 pre_merge['area']랑 왜 다르냐?
# def area_words(word):
#         #1
#     if word == '전북':
#       word = '전북'
#         #2
#     elif word == '제주':
#       word = '제주'
#         #3
#     elif word == '충남':
#       word = '충남'
#         #4
#     elif word == '충북':
#       word = '충북'
#         #5
#     elif word == '강원':
#       word = '강원'
#         #6
#     elif word == '경기':
#       word = '경기'
#         #7
#     elif word == '경남':
#       word = '경남'
#         #8
#     elif word == '경북':
#       word = '경북'
#         #9
#     elif word == '광주':
#       word = '광주'
#         #10
#     elif word == '대구':
#       word = '대구'
#         #11
#     elif word == '대전':
#       word = '대전'
#         #12
#     elif word == '부산':
#       word = '부산'
#         #13
#     elif word == '서울':
#       word = '서울'
#         #14
#     elif word == '세종':
#       word = '세종'
#         #15
#     elif word == '울산':
#       word = '울산'
#         #16
#     elif word == '인천':
#       word = '인천'
#         #17
#     elif word == '전남':
#       word = '전남'

#     return word
# pre_merge['area'] = [area_words(i) for i in pre_merge['area']]
# back = pd.merge(back, pre_merge, how='outer', on =['yyyymmdd','area'])

# # back.to_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/fct_weather_merge.csv',encoding='cp949')

  0%|          | 0/7 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
2012 3시간기온


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: FutureWarning: Passing 'suffixes' which cause duplicate columns {'val_x'} in the result is deprecated and will raise a MergeError in a future version.


광주
대구
경북
세종
울산
서울
부산
인천
전남
전북
제주
충남
충북
강원
경기
경남
대전


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
2016 적설


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: FutureWarning: Passing 'suffixes' which cause duplicate columns {'val_x'} in the result is deprecated and will raise a MergeError in a future version.


강원
경기
경남
경북
광주
대구
대전
부산
서울
세종
울산
인천
전남
전북
제주
충남
충북


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 7/7 [06:00<00:00, 51.49s/it]


In [1]:
fct_final[0].isna().sum()

NameError: ignored

In [194]:
fct_final[1].isna().sum()

습도_mean_9     0
습도_mean_12    0
습도_mean_15    0
습도_mean_33    0
습도_mean_36    0
습도_mean_39    0
area             0
yyyymmdd         0
dtype: int64

In [205]:
fct_final[2].isna().sum()

일최고기온_mean_9         0
일최고기온_mean_12        0
일최고기온_mean_15        0
area                       85
일최고기온_mean_33    11822
yyyymmdd                    0
dtype: int64

In [ ]:
fct_final[3].isna().sum()

In [197]:
fct_final[4].isna().sum()

풍속_mean_9     0
풍속_mean_12    0
풍속_mean_15    0
풍속_mean_33    0
풍속_mean_36    0
풍속_mean_39    0
area              0
yyyymmdd          0
dtype: int64

In [198]:
fct_final[5].isna().sum()

강수량_mean_9        0
강수량_mean_12    8870
강수량_mean_15       0
강수량_mean_33       0
강수량_mean_36    8870
강수량_mean_39       0
area                  51
yyyymmdd               0
dtype: int64

In [200]:
fct_final[6].isna().sum()

적설_mean_9        0
적설_mean_12    2612
적설_mean_15       0
적설_mean_33       0
적설_mean_36    2612
적설_mean_39       0
area                51
yyyymmdd             0
dtype: int64

In [36]:
# 백병원데이터
back = pd.read_csv('/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/final/final_res.csv',encoding='cp949')
back.drop(back.columns[4:],axis=1,inplace=True)


def area_words(word):
        #1
    if word == '전북':
      word = '전북'
        #2
    elif word == '제주':
      word = '제주'
        #3
    elif word == '충남':
      word = '충남'
        #4
    elif word == '충북':
      word = '충북'
        #5
    elif word == '강원':
      word = '강원'
        #6
    elif word == '경기':
      word = '경기'
        #7
    elif word == '경남':
      word = '경남'
        #8
    elif word == '경북':
      word = '경북'
        #9
    elif word == '광주':
      word = '광주'
        #10
    elif word == '대구':
      word = '대구'
        #11
    elif word == '대전':
      word = '대전'
        #12
    elif word == '부산':
      word = '부산'
        #13
    elif word == '서울':
      word = '서울'
        #14
    elif word == '세종':
      word = '세종'
        #15
    elif word == '울산':
      word = '울산'
        #16
    elif word == '인천':
      word = '인천'
        #17
    elif word == '전남':
      word = '전남'

    return word
pre_merge['area'] = [area_words(i) for i in pre_merge['area']]
back = pd.merge(back, pre_merge, how='outer', on =['yyyymmdd','area'])

In [127]:
back_cpy = back.sort_values(['area','yyyymmdd']).copy()
back_cpy['yyyymmdd'] = pd.to_datetime(back_cpy['yyyymmdd'],format='%Y%m%d')
back_cpy.set_index('yyyymmdd',inplace=True)
back_cpy.iloc[:,3:] = back_cpy.iloc[:,3:].interpolate(method='time')



back_cpy['yyyymmdd'] = back_cpy.index
back_cpy['sex'].unique()
back_cpy.index = [i for i in range(len(back_cpy))] 

back_cpy.sort_values(['yyyymmdd','area'],inplace=True)

back_cpy['yyyymmdd'] = back_cpy['yyyymmdd'].astype(str)
back_cpy['yyyymmdd'] = back_cpy['yyyymmdd'].str.replace(pat=r'-', repl=r'', regex=True)
back_cpy['yyyymmdd'] = back_cpy['yyyymmdd'].astype('int')

back_cpy = back_cpy[back.columns]

back_cpy.reset_index(drop=True,inplace=True)

In [130]:
pre_merge

,3시간기온_mean_9,3시간기온_mean_12,3시간기온_mean_15,3시간기온_mean_33,3시간기온_mean_36,3시간기온_mean_39,area,yyyymmdd,습도_mean_9,습도_mean_12,...,12시간강수량_mean_15,12시간강수량_mean_33,12시간강수량_mean_36,12시간강수량_mean_39,12시간신적설_mean_9,12시간신적설_mean_12,12시간신적설_mean_15,12시간신적설_mean_33,12시간신적설_mean_36,12시간신적설_mean_39
0,-0.333333,-2.655556,-4.605556,-0.275000,-1.913889,-3.102778,전북,20120101,-0.333333,-2.655556,...,-4.605556,-0.275000,-1.913889,-3.102778,-0.333333,-2.655556,-4.605556,-0.275000,-1.913889,-3.102778
1,-0.055556,-1.985185,-3.133333,-0.275000,-1.913889,-3.102778,전북,20120102,-0.055556,-1.985185,...,-3.133333,-0.275000,-1.913889,-3.102778,-0.055556,-1.985185,-3.133333,-0.275000,-1.913889,-3.102778
2,-0.133333,-1.740741,-3.050000,0.169444,-1.216667,-2.575000,전북,20120103,-0.133333,-1.740741,...,-3.050000,0.169444,-1.216667,-2.575000,-0.133333,-1.740741,-3.050000,0.169444,-1.216667,-2.575000
3,-3.494444,-5.088889,-6.438889,-2.702778,-3.588889,-4.605556,전북,20120104,-3.494444,-5.088889,...,-6.438889,-2.702778,-3.588889,-4.605556,-3.494444,-5.088889,-6.438889,-2.702778,-3.588889,-4.605556
4,-2.250000,-4.707407,-6.363889,-2.058333,-4.391667,-5.677778,전북,20120105,-2.250000,-4.707407,...,-6.363889,-2.058333,-4.391667,-5.677778,-2.250000,-4.707407,-6.363889,-2.058333,-4.391667,-5.677778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6147,2.084615,1.189744,1.068910,2.065385,1.073077,0.941346,전남,20121227,2.084615,1.189744,...,1.068910,2.065385,1.073077,0.941346,2.084615,1.189744,1.068910,2.065385,1.073077,0.941346
6148,2.476923,1.300000,0.396154,3.400962,2.036538,0.701923,전남,20121228,2.476923,1.300000,...,0.396154,3.400962,2.036538,0.701923,2.476923,1.300000,0.396154,3.400962,2.036538,0.701923
6149,3.115385,2.198291,1.373164,3.509615,2.424679,1.670994,전남,20121229,3.115385,2.198291,...,1.373164,3.509615,2.424679,1.670994,3.115385,2.198291,1.373164,3.509615,2.424679,1.670994
6150,-1.230769,-2.392308,-3.073077,-1.471154,-2.383654,-2.867308,전남,20121230,-1.230769,-2.392308,...,-3.073077,-1.471154,-2.383654,-2.867308,-1.230769,-2.392308,-3.073077,-1.471154,-2.383654,-2.867308
